In [ ]:
!pip install azure-cli
!pip install pydicom azure.identity
!pip install requests_toolbelt

In [ ]:
import requests
import pydicom
from pathlib import Path
import matplotlib.pylab as plt
from azure.identity import DefaultAzureCredential
from urllib3.filepost import encode_multipart_formdata, choose_boundary
import requests_toolbelt as tb
from io import BytesIO
import pandas as pd
from PIL import Image

In [ ]:
base_url = f"https://mammogram-mammogram.dicom.azurehealthcareapis.com/v1"

In [ ]:
mass_train=pd.read_csv('/content/drive/MyDrive/mass_case_description_train_set.csv')
mass_train['studyid']=mass_train['image file path'].apply(lambda x: x.split('/')[1])
mass_train['seriesid']=mass_train['image file path'].apply(lambda x: x.split('/')[2])
feature_label_train_mass=mass_train[['studyid','pathology']]

In [ ]:
def azlogin():
  !az login

In [ ]:
def get_cred_and_check_auth():
  

  credential = DefaultAzureCredential()

  #print(credential.credentials) # this can be used to find the index of the AzureCliCredential
  token = credential.credentials[4].get_token('https://dicom.healthcareapis.azure.com')
  bearer_token = f'Bearer {token.token}'
  def encode_multipart_related(fields, boundary=None):
      if boundary is None:
          boundary = choose_boundary()

      body, _ = encode_multipart_formdata(fields, boundary)
      content_type = str('multipart/related; boundary=%s' % boundary)

      return body, content_type
    
  client = requests.session()

  headers = {"Authorization":bearer_token}
  url= f'{base_url}/changefeed'

  response = client.get(url,headers=headers)
  if (response.status_code != 200):
      print('Error! Likely not authenticated!')
  else:
    print("Success")
  return bearer_token,client

In [ ]:
azlogin()

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EJXYCXCAV to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "e056658c-7855-40e3-9a64-9a8c4fe82c9a",
    "id": "c7518a1a-6651-45cb-8726-8fd7cf93e0cb",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure for Students",
    "state": "Enabled",
    "tenantId": "e056658c-7855-40e3-9a64-9a8c4fe82c9a",
    "user": {
      "name": "spp26041999@gmail.com",
      "type": "user"
    }
  }
]


In [ ]:
bearer_token, client = get_cred_and_check_auth()

Success


In [ ]:
def image_fetch(studyid_idx):
  url = f'{base_url}/studies/{feature_label_train_mass.studyid[studyid_idx]}'
  headers = {'Accept':'multipart/related; type="application/dicom"; transfer-syntax=*', "Authorization":bearer_token}

  response = client.get(url, headers=headers) #, verify=False)
  mpd = tb.MultipartDecoder.from_response(response)

  mpd = tb.MultipartDecoder.from_response(response)
  part=mpd.parts[0]
  ds = pydicom.dcmread(BytesIO(part.content))
  return ds.pixel_array

In [ ]:
image_fetch(n)

NonMultipartContentTypeException: ignored

In [1]:
n=999

Image.fromarray(image_fetch(n)).save(f"{feature_label_train_mass.studyid[n]}.png")

NameError: ignored

In [ ]:
##Loop To Transfer DICOM TO DRIVE## DONT DELETE
for i in range(1100,1111):#1318):#(1007)
  Image.fromarray(image_fetch(i)).save(f"{feature_label_train_mass.studyid[i]}.png")
  print(i)

NonMultipartContentTypeException: ignored